# Generate UniRes bash file

In [ ]:
import os

# Specify the folder path where your files are located
folder_path = '/code/imagesTr'

# List all files in the folder
files = os.listdir(folder_path)

# Create or open a text file for writing
with open('/code/output.txt', 'w') as txt_file:
    # Iterate through the files and write each line
    for file_name in files:
        txt_file.write(f'unires --linear {file_name}\n')

# upsample the UniRes output image

In [2]:
import os
import numpy as np
import SimpleITK as sitk
from skimage.transform import resize

# Input and output directories
input_dir = '/code/unires_og'
output_dir = '/code/unires_upsample'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define the target size (width and height) for upsampling
target_size = 512

# Loop through all the files in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith('.nii.gz'):
        # Load the NIfTI file using SimpleITK
        file_path = os.path.join(input_dir, file_name)
        image = sitk.ReadImage(file_path)

        # Upsample the image data using scikit-image
        image_data = sitk.GetArrayFromImage(image)

        # Get original dimensions
        original_depth, original_width, original_height = image_data.shape

        print(original_depth, original_width, original_height )

        # Calculate the scaling factor for the depth
        scaling_factor = target_size / original_height

        print(scaling_factor)


        new_shape = (
            int(original_depth * scaling_factor),
            target_size,
            target_size
        )

        print(new_shape)

        upsampled_data = resize(image_data, new_shape, order=1, preserve_range=True)

        # Create a new SimpleITK image with the upsampled data
        upsampled_image = sitk.GetImageFromArray(upsampled_data)

        # Set the spacing, direction, and origin
        spacing = (
            image.GetSpacing()[0]/scaling_factor,
            image.GetSpacing()[1]/scaling_factor,
            image.GetSpacing()[2]/scaling_factor
        )

        print(spacing)        
        upsampled_image.SetSpacing(spacing)
        upsampled_image.SetDirection(image.GetDirection())
        upsampled_image.SetOrigin(image.GetOrigin())

        # Save the upsampled image to the output directory
        output_file_path = os.path.join(output_dir, file_name)
        sitk.WriteImage(upsampled_image, output_file_path)

        print(f"Upsampled and saved: {output_file_path}")



217 293 290
1.7655172413793103
(383, 512, 512)
(0.56640625, 0.56640625, 0.56640625)
Upsampled and saved: /code/unires_upsample/ur_CQ500CT4_5mm_0_2.nii.gz


# Duplicate the labels to align the shape of upsampled UniRes

In [4]:
import itk
import os
import numpy as np

thin_folder = '/code/imagesTr_linear_upsample/'
thick_folder = '/code/labelsTr/'
output_folder = '/code/labelsTr_linear_aligned/'

# Load the thin and thick images
thin_images = sorted(os.listdir(thin_folder))
thick_images = sorted(os.listdir(thick_folder))

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all files in the thin folder
thin_files = os.listdir(thin_folder)

# Iterate through each thin file
for thin_file in thin_files:
    # Construct the paths to thin and thick files
    thin_path = os.path.join(thin_folder, thin_file)
    thick_path = os.path.join(thick_folder, thin_file)

    # Load the thin and thick NIfTI images
    thin_img = itk.array_view_from_image(itk.imread(thin_path))
    thick_img = itk.array_view_from_image(itk.imread(thick_path))

    # Determine the number of slices in each image
    num_thin_slices = thin_img.shape[0]
    num_thick_slices = thick_img.shape[0]

    # Calculate the duplication factor
    duplication_factor = int(num_thin_slices / num_thick_slices)

    # Duplicate the thick slices
    duplicated_thick = np.repeat(thick_img, duplication_factor, axis=0)


    # If the duplicated thick is still less than thin, copy and paste the last slice
    if duplicated_thick.shape[0] < num_thin_slices:
        last_slice = duplicated_thick[-1]
        num_slices_to_add = num_thin_slices - duplicated_thick.shape[0]
        duplicated_thick = np.concatenate([duplicated_thick, np.repeat(last_slice[np.newaxis, :, :], num_slices_to_add, axis=0)], axis=0)

    duplicated_thick = duplicated_thick[:, :, ::-1]
    # Save the duplicated thick image to the output folder
    output_path = os.path.join(output_folder, thin_file)
    duplicated_thick_volume = itk.GetImageFromArray(duplicated_thick.astype(np.uint8))
    itk.imwrite(duplicated_thick_volume, output_path)

    # Print the shapes of thin, thick, and duplicated thick images
    print(f"Thin shape: {thin_img.shape}")
    print(f"Thick shape: {thick_img.shape}")
    print(f"Duplicated Thick shape: {duplicated_thick.shape}")


Thin shape: (281, 512, 512)
Thick shape: (28, 512, 512)
Duplicated Thick shape: (281, 512, 512)
Thin shape: (340, 512, 512)
Thick shape: (32, 512, 512)
Duplicated Thick shape: (340, 512, 512)
Thin shape: (348, 512, 512)
Thick shape: (35, 512, 512)
Duplicated Thick shape: (348, 512, 512)
Thin shape: (313, 512, 512)
Thick shape: (32, 512, 512)
Duplicated Thick shape: (313, 512, 512)
Thin shape: (324, 512, 512)
Thick shape: (30, 512, 512)
Duplicated Thick shape: (324, 512, 512)
Thin shape: (363, 512, 512)
Thick shape: (36, 512, 512)
Duplicated Thick shape: (363, 512, 512)
Thin shape: (283, 512, 512)
Thick shape: (28, 512, 512)
Duplicated Thick shape: (283, 512, 512)
Thin shape: (433, 512, 512)
Thick shape: (44, 512, 512)
Duplicated Thick shape: (433, 512, 512)
Thin shape: (208, 512, 512)
Thick shape: (40, 512, 512)
Duplicated Thick shape: (208, 512, 512)
Thin shape: (379, 512, 512)
Thick shape: (36, 512, 512)
Duplicated Thick shape: (379, 512, 512)
Thin shape: (340, 512, 512)
Thick shape: